# DS 6050 Lung Cancer Classification
## Thomas Butler, vra2cf
## Drew Haynes, rbc6wr
## Christian Schroeder, dbn5eu

# Load Packages

In [ ]:
import sys
import sklearn
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
!pip install SimpleITK
import SimpleITK as sitk
import csv
from PIL import Image
%matplotlib inline

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.8.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 46.9 MB/s 


# Import Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = "/content/drive/MyDrive/LUNA/"

Ref: https://luna16.grand-challenge.org/Tutorial/

Used this code to help preprocess data into a format we can use.

We define now a function to:
    - Open the image 
    - Store it into a numpy array
    - Extract the following info: Pixel Spacing, Origin
This function takes as input the name of the image and returns:
    - The array corresponding to the image (numpyImage)
    - Origin (numpyOrigin)
    - PixelSpacing (numpySpacing)

In [ ]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
     
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
     
    return numpyImage, numpyOrigin, numpySpacing

To be able to open and read the list of candidates, we need to use the csv python module. 
We define now a function to:
    - Open a csv file
    - Read a csv file
    - Save each line of a csv file
This functions takes as input the name of the csv file and returns:
    - A list of each line of the csv

def readCSV(filename):
    lines = []
    with open(filename, "rt") as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            lines.append(line)
    return lines

Since the coordinates of the candidates are given in World Coordinates, we now need to transform from world coordinates to voxel coordinates. 
We define now a function to do that. Please note that the transformation below is only valid if there is no rotation component in the transformation matrix. For all CT images in our dataset, there is no rotation component so that this formula can be used. 
This function takes as inputs:
    - The world coordinates
    - The origin
    - The pixel Spacing
This function returns:
    - Voxel coordinates (voxelCoord)

In [ ]:
def worldToVoxelCoord(worldCoord, origin, spacing):
     
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

We want to extract now some features from the candidates. We define some normalized planes to extract views from the candidates

In [ ]:
def normalizePlanes(npzarray):
     
    maxHU = 400.
    minHU = -1000.
 
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray

    - Specify the path where the image (img_path) is 
    - Specificy the path where the file with the list of candidates is (cand_path)

In [ ]:
cand_path = root+'candidates_V2/candidates_V2.csv'

In [ ]:
subsets = range(10)
img_paths = []
for subset in subsets : # Doing this to loop instead of a general recursive
                        # loop to 1) avoid segs_lungs_LUNA16 files and 2) to 
                        # allow flexibility with which subsets to use if desired
  subset_root = root+'subset'+str(subset)+'/'
  for img_path in os.listdir(subset_root):
        if img_path.endswith('.mhd'):
          img_paths.append(subset_root+img_path)


    - Load the image
    - Extract the Origin
    - Extract the Pixel Spacing 

In [ ]:
# load image
numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_paths[0])
print(numpyImage.shape)
print(numpyOrigin)
print(numpySpacing)

(121, 512, 512)
[-335.209991 -195.       -198.100006]
[2.5        0.76171899 0.76171899]


    - Load the csv file
    - Get the candidates 
    - Transform from world to voxel coordinates

In [ ]:
# load candidates
cands_master_df = pd.read_csv(cand_path)

In [ ]:
#subset dataset.
df_1 =cands_master_df[cands_master_df['class'] == 1]
df_0 =cands_master_df[cands_master_df['class'] == 0].sample(frac=1).head(10000)
cands_master_df = pd.concat([df_1,df_0]).sample(frac=1)

In [ ]:
# List of each unique SeriesUID, i.e. candidates
cands = cands_master_df.seriesuid.unique()

# Dictionary 
cands_df_dict = {elem : pd.DataFrame() for elem in cands}

for key in cands_df_dict.keys():
    cands_df_dict[key] = cands_master_df[cands_master_df.seriesuid == key]
    cands_df_dict[key] = cands_df_dict[key].drop(['seriesuid'], axis=1)

In [ ]:
Fail_save = 0
Fail_convert = 0
Could_not_find = 0
if not os.path.exists(root+'data_subset/'):
  os.mkdir(root+'data_subset/')
if not os.path.exists(root+'data_subset/0/'):
  os.mkdir(root+'data_subset/0/')
if not os.path.exists(root+'data_subset/1/'):
  os.mkdir(root+'data_subset/1/')
if not os.path.exists(root+'patches/'):
  os.mkdir(root+'patches/')

for cand_id in cands_df_dict:
  try:
    voxelWidth = 64
    cand_image_path = [i for i in img_paths if cand_id in i][0]
    numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_paths[0])
    points_for_cand = cands_df_dict[cand_id]
    for index, point in points_for_cand.iterrows():
        #print(point[2])
        worldCoord = np.asarray([float(point[2]),float(point[1]),float(point[0])])
        voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
        try : 
          patch = numpyImage[int(voxelCoord[0]),int(voxelCoord[1]-voxelWidth/2):int(voxelCoord[1]+voxelWidth/2),int(voxelCoord[2]-voxelWidth/2):int(voxelCoord[2]+voxelWidth/2)]
          patch = normalizePlanes(patch)
          #print('data')
          #print(worldCoord)
          #print(voxelCoord)
          cand_root = root+'patches/'+cand_id+'/'
          if not os.path.exists(cand_root):
            os.mkdir(cand_root)
          #outputPath = cand_root + 'patch_' + str(worldCoord[0]) + '_' + str(worldCoord[1]) + '_' + str(worldCoord[2]) + str(int(point[3])) + '.tiff'
          #save jpg under data and class 0 or 1
          outputPath = root + 'data_subset/' + str(int(point[3])) + '/patch_' + str(worldCoord[0]) + '_' + str(worldCoord[1]) + '_' + str(worldCoord[2]) + str(int(point[3])) + '.jpg'
          #plt.imshow(patch, cmap='gray')
          #plt.show()
          f = open(outputPath, "w")
          f.close()
          try:
            patch_image = Image.fromarray(patch*255).convert('L')
            #patch_image = patch_image.save(outputPath)
            #save jpg keep image the same.
            patch_image = patch_image.save(outputPath)
          except:
            print(f'Failed to save {outputPath}')
            Fail_save+=1
            os.remove(outputPath)
        except :
          print(f'Failed to convert point: ID: {cand_id}, Index: {index}')
          Fail_convert+=1
  except:
    print(f'Could not find {cand_id}')
    Could_not_find+=1

      

Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.309955999522338651429118207446, Index: 511250
Could not find 1.3.6.1.4.1.14519.5.2.1.6279.6001.320967206808467952819309001585
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.259227883564429312164962953756, Index: 401913
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680, Index: 434711
Failed to save /content/drive/MyDrive/LUNA/data_subset/0/patch_-625.59_228.33_-68.050.jpg
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680, Index: 434583
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680, Index: 434036
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680, Index: 434860
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680, Index: 433934
Failed to convert point: ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.2735

In [ ]:
print(Fail_save)
print(Fail_convert)
print(Could_not_find)

346
666
132


In [ ]:
import pandas as pd

cand_csv = pd.read_csv(cand_path)
cand_csv['class'].value_counts()

0    753418
1      1557
Name: class, dtype: int64

In [ ]:
from os import listdir
file_names_0 = [file for file in os.listdir(root+'data_subset/0')]
len(file_names_0)

7570

In [ ]:
file_names_1 = [file for file in os.listdir(root+'data_subset/1')]
len(file_names_1)

1184

In [ ]:
from sklearn.datasets import load_files 
from keras.utils import np_utils

from keras.preprocessing import image
from tqdm import tqdm # progress bar

data_dir = root + "data_subset/"
batch_size = 128;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 64; 
img_width = 64;

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'grayscale',
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'grayscale',
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)        


Found 8754 files belonging to 2 classes.
Using 7004 files for training.
Found 8754 files belonging to 2 classes.
Using 1750 files for validation.


In [ ]:
tf.random.set_seed(42)
#train is 70% of data, test is 10% of data.
#ref: https://stackoverflow.com/questions/50737192/tf-data-dataset-how-to-get-the-dataset-size-number-of-elements-in-an-epoch
dataset_size = train_ds.cardinality().numpy()
train_size = int(0.875 * dataset_size)
test_size = int(0.125 * dataset_size)

temp_ds = train_ds.shuffle(1000, seed=42)
train_ds = temp_ds.take(train_size)
test_ds = temp_ds.skip(train_size)

## Metrics Functions

In [ ]:
#ref: https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d

import keras.backend as K

def Precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def Recall(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def F1_score(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val

In [ ]:
#keras.metrics.AUC(name='prc', curve='PR'), keras.metrics.AUC(curve='PR')

#ref:https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
  return tf.py_function(roc_auc_score, (y_true, y_pred[:,1]), tf.double)

#below function doesn't work with current code.
#def auc(y_true, y_pred):
#  auc = tf.keras.metrics.AUC(y_true, y_pred[:,1])[1]
#  K.get_session().run(tf.local_variables_initializer())
#  return auc

# Base Model

In [ ]:
from functools import partial

tf.keras.backend.clear_session()

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=4, activation='relu', padding="SAME")

model = keras.models.Sequential([
    keras.Input(shape=[64, 64, 1]),
    keras.layers.BatchNormalization(),
    #DefaultConv2D(filters=32, kernel_size=8),
    #DefaultConv2D(filters=32, kernel_size=8),
    #keras.layers.BatchNormalization(),
    #keras.layers.MaxPooling2D(pool_size=2),
    #DefaultConv2D(filters=32),
    #DefaultConv2D(filters=32),
    #keras.layers.BatchNormalization(),
    #keras.layers.MaxPooling2D(pool_size=2), 
    DefaultConv2D(filters=16),
    #DefaultConv2D(filters=16),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    #keras.layers.Dense(units=64, activation='relu'),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(units=2, activation='softmax'),
])

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy", F1_score, Precision, Recall, auroc])
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)
score = model.evaluate(test_ds)

Epoch 1/10
48/48 [==============================] - 21s 41ms/step - loss: 0.8724 - accuracy: 0.7880 - F1_score: 0.2438 - Precision: 0.1395 - Recall: 1.0000 - auroc: 0.4943 - val_loss: 0.5464 - val_accuracy: 0.8257 - val_F1_score: 0.2264 - val_Precision: 0.1288 - val_Recall: 1.0000 - val_auroc: 0.5354
Epoch 2/10
48/48 [==============================] - 6s 34ms/step - loss: 0.4620 - accuracy: 0.8477 - F1_score: 0.2369 - Precision: 0.1353 - Recall: 1.0000 - auroc: 0.5678 - val_loss: 0.5404 - val_accuracy: 0.8606 - val_F1_score: 0.2287 - val_Precision: 0.1299 - val_Recall: 1.0000 - val_auroc: 0.5212
Epoch 3/10
48/48 [==============================] - 6s 34ms/step - loss: 0.4333 - accuracy: 0.8538 - F1_score: 0.2364 - Precision: 0.1349 - Recall: 1.0000 - auroc: 0.5993 - val_loss: 0.4661 - val_accuracy: 0.8669 - val_F1_score: 0.2263 - val_Precision: 0.1283 - val_Recall: 1.0000 - val_auroc: 0.5503
Epoch 4/10
48/48 [==============================] - 6s 35ms/step - loss: 0.4155 - accuracy: 0.85

# Test Model

Note I did alot of iterations based on commented out code. Basically every combination of the commented out code including increasing filters or dense units by a factor of 2.

In [ ]:
from functools import partial

tf.keras.backend.clear_session()

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=4, activation='relu', padding="SAME")

model = keras.models.Sequential([
    keras.Input(shape=[64, 64, 1]),
    keras.layers.BatchNormalization(),
    DefaultConv2D(filters=32, kernel_size=8),
    #DefaultConv2D(filters=32, kernel_size=8),
    #keras.layers.BatchNormalization(),
    #keras.layers.MaxPooling2D(pool_size=2),
    #DefaultConv2D(filters=32),
    #DefaultConv2D(filters=32),
    #keras.layers.BatchNormalization(),
    #keras.layers.MaxPooling2D(pool_size=2), 
    DefaultConv2D(filters=16),
    #DefaultConv2D(filters=16),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    #keras.layers.Dense(units=64, activation='relu'),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(units=2, activation='softmax'),
])

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy", F1_score, Precision, Recall, auroc])
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)
score = model.evaluate(test_ds)

Epoch 1/10
48/48 [==============================] - 8s 57ms/step - loss: 0.5931 - accuracy: 0.8206 - F1_score: 0.2386 - Precision: 0.1364 - Recall: 1.0000 - auroc: 0.5280 - val_loss: 0.5808 - val_accuracy: 0.8714 - val_F1_score: 0.2273 - val_Precision: 0.1291 - val_Recall: 1.0000 - val_auroc: 0.5263
Epoch 2/10
48/48 [==============================] - 7s 52ms/step - loss: 0.4024 - accuracy: 0.8603 - F1_score: 0.2385 - Precision: 0.1363 - Recall: 1.0000 - auroc: 0.6298 - val_loss: 0.5721 - val_accuracy: 0.8714 - val_F1_score: 0.2285 - val_Precision: 0.1296 - val_Recall: 1.0000 - val_auroc: 0.5441
Epoch 3/10
48/48 [==============================] - 7s 50ms/step - loss: 0.3761 - accuracy: 0.8631 - F1_score: 0.2372 - Precision: 0.1352 - Recall: 1.0000 - auroc: 0.6920 - val_loss: 0.6087 - val_accuracy: 0.8674 - val_F1_score: 0.2277 - val_Precision: 0.1294 - val_Recall: 1.0000 - val_auroc: 0.5599
Epoch 4/10
48/48 [==============================] - 7s 50ms/step - loss: 0.3497 - accuracy: 0.871

## Xception Model

In [ ]:
data_dir = root + "data_subset/"
batch_size = 32;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 299; 
img_width = 299;

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'rgb',
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'rgb',
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)        


Found 6568 files belonging to 2 classes.
Using 5255 files for training.
Found 6568 files belonging to 2 classes.
Using 1313 files for validation.


In [ ]:
tf.random.set_seed(42)
#train is 70% of data, test is 10% of data.
#ref: https://stackoverflow.com/questions/50737192/tf-data-dataset-how-to-get-the-dataset-size-number-of-elements-in-an-epoch
dataset_size = train_ds.cardinality().numpy()
train_size = int(0.875 * dataset_size)
test_size = int(0.125 * dataset_size)

temp_ds = train_ds.shuffle(1000, seed=42)
train_ds = temp_ds.take(train_size)
test_ds = temp_ds.skip(train_size)

In [ ]:
n_classes = 2
def xception_preprocess_x(x, y):
  x = tf.keras.applications.xception.preprocess_input(x)
  return x,y
xception_ds_train = train_ds.map(xception_preprocess_x)
xception_ds_val = validation_ds.map(xception_preprocess_x)
xception_ds_test = test_ds.map(xception_preprocess_x)

In [ ]:
#ref: https://stackoverflow.com/questions/50737192/tf-data-dataset-how-to-get-the-dataset-size-number-of-elements-in-an-epoch
dataset_size = xception_ds_train.cardinality().numpy()
dataset_size

144

In [ ]:
tf.keras.backend.clear_session()
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

83697664/83683744 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(xception_ds_train,
                    validation_data=xception_ds_val,
                    epochs=5)

Epoch 1/5
144/144 [==============================] - 171s 368ms/step - loss: 1.4358 - accuracy: 0.7785 - F1_score: 0.2393 - Precision: 0.1394 - Recall: 1.0000 - val_loss: 0.7293 - val_accuracy: 0.8416 - val_F1_score: 0.2554 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 2/5
144/144 [==============================] - 52s 321ms/step - loss: 0.8102 - accuracy: 0.8006 - F1_score: 0.2339 - Precision: 0.1354 - Recall: 0.9931 - val_loss: 0.6157 - val_accuracy: 0.8180 - val_F1_score: 0.2551 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 3/5
144/144 [==============================] - 54s 335ms/step - loss: 0.4444 - accuracy: 0.8377 - F1_score: 0.2425 - Precision: 0.1413 - Recall: 1.0000 - val_loss: 0.5808 - val_accuracy: 0.8462 - val_F1_score: 0.2533 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 4/5
144/144 [==============================] - 55s 343ms/step - loss: 0.3665 - accuracy: 0.8529 - F1_score: 0.2339 - Precision: 0.1360 - Recall: 0.9931 - val_loss: 0.5065 - val_accuracy: 0.

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(xception_ds_train,
                    validation_data=xception_ds_val,
                    epochs=10)

Epoch 1/10
144/144 [==============================] - 193s 1s/step - loss: 0.4131 - accuracy: 0.8595 - F1_score: 0.2322 - Precision: 0.1345 - Recall: 0.9931 - val_loss: 0.4517 - val_accuracy: 0.8469 - val_F1_score: 0.2542 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 2/10
144/144 [==============================] - 187s 1s/step - loss: 0.3534 - accuracy: 0.8622 - F1_score: 0.2398 - Precision: 0.1395 - Recall: 0.9861 - val_loss: 0.4327 - val_accuracy: 0.8393 - val_F1_score: 0.2787 - val_Precision: 0.1726 - val_Recall: 1.0000
Epoch 3/10
144/144 [==============================] - 186s 1s/step - loss: 0.2756 - accuracy: 0.8905 - F1_score: 0.2394 - Precision: 0.1394 - Recall: 0.9931 - val_loss: 0.4935 - val_accuracy: 0.8165 - val_F1_score: 0.2542 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 4/10
144/144 [==============================] - 186s 1s/step - loss: 0.1888 - accuracy: 0.9286 - F1_score: 0.2416 - Precision: 0.1404 - Recall: 0.9931 - val_loss: 0.6545 - val_accuracy: 0.7822 

In [ ]:
score = model.evaluate(xception_ds_test)

21/21 [==============================] - 11s 269ms/step - loss: 0.0029 - accuracy: 1.0000 - F1_score: 0.2439 - Precision: 0.1414 - Recall: 1.0000


## InceptionResNetV2 Model

In [ ]:
n_classes = 2
def InceptionResNetV2_preprocess_x(x, y):
  x = tf.keras.applications.inception_resnet_v2.preprocess_input(x)
  return x,y
InceptionResNetV2_ds_train = train_ds.map(InceptionResNetV2_preprocess_x)
InceptionResNetV2_ds_val = validation_ds.map(InceptionResNetV2_preprocess_x)
InceptionResNetV2_ds_test = test_ds.map(InceptionResNetV2_preprocess_x)

In [ ]:
tf.keras.backend.clear_session()
base_model = keras.applications.InceptionResNetV2(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(InceptionResNetV2_ds_train,
                    validation_data=InceptionResNetV2_ds_val,
                    epochs=10)

Epoch 1/10
144/144 [==============================] - 60s 260ms/step - loss: 5.0471 - accuracy: 0.7694 - F1_score: 0.2397 - Precision: 0.1391 - Recall: 0.9931 - val_loss: 6.3498 - val_accuracy: 0.4433 - val_F1_score: 0.2551 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 2/10
144/144 [==============================] - 41s 235ms/step - loss: 2.4340 - accuracy: 0.7759 - F1_score: 0.2387 - Precision: 0.1387 - Recall: 1.0000 - val_loss: 3.2758 - val_accuracy: 0.8469 - val_F1_score: 0.2533 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 3/10
144/144 [==============================] - 41s 233ms/step - loss: 2.5297 - accuracy: 0.7805 - F1_score: 0.2356 - Precision: 0.1368 - Recall: 0.9931 - val_loss: 0.8781 - val_accuracy: 0.8088 - val_F1_score: 0.2548 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 4/10
144/144 [==============================] - 41s 234ms/step - loss: 1.0305 - accuracy: 0.8036 - F1_score: 0.2371 - Precision: 0.1375 - Recall: 1.0000 - val_loss: 1.7129 - val_accuracy:

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(InceptionResNetV2_ds_train,
                    validation_data=InceptionResNetV2_ds_val,
                    epochs=20)

Epoch 1/20
144/144 [==============================] - 121s 640ms/step - loss: 0.5875 - accuracy: 0.8414 - F1_score: 0.2371 - Precision: 0.1387 - Recall: 0.9792 - val_loss: 1.8299 - val_accuracy: 0.6885 - val_F1_score: 0.2784 - val_Precision: 0.1726 - val_Recall: 1.0000
Epoch 2/20
144/144 [==============================] - 95s 610ms/step - loss: 0.3992 - accuracy: 0.8636 - F1_score: 0.2350 - Precision: 0.1364 - Recall: 0.9931 - val_loss: 0.5808 - val_accuracy: 0.8050 - val_F1_score: 0.2544 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 3/20
144/144 [==============================] - 95s 610ms/step - loss: 0.3888 - accuracy: 0.8634 - F1_score: 0.2356 - Precision: 0.1365 - Recall: 1.0000 - val_loss: 0.4141 - val_accuracy: 0.8469 - val_F1_score: 0.2550 - val_Precision: 0.1496 - val_Recall: 0.9762
Epoch 4/20
144/144 [==============================] - 96s 610ms/step - loss: 0.3879 - accuracy: 0.8617 - F1_score: 0.2373 - Precision: 0.1377 - Recall: 0.9931 - val_loss: 0.4094 - val_accuracy

In [ ]:
score = model.evaluate(InceptionResNetV2_ds_test)

21/21 [==============================] - 11s 182ms/step - loss: 0.1231 - accuracy: 0.9405 - F1_score: 0.2560 - Precision: 0.1548 - Recall: 0.9524


## ResNet152V2 Model



In [ ]:
data_dir = root + "data_subset/"
batch_size = 64;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 224; 
img_width = 224;

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'rgb',
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    color_mode = 'rgb',
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)        

Found 6568 files belonging to 2 classes.
Using 5255 files for training.
Found 6568 files belonging to 2 classes.
Using 1313 files for validation.


In [ ]:
tf.random.set_seed(42)
#train is 70% of data, test is 10% of data.
#ref: https://stackoverflow.com/questions/50737192/tf-data-dataset-how-to-get-the-dataset-size-number-of-elements-in-an-epoch
dataset_size = train_ds.cardinality().numpy()
train_size = int(0.875 * dataset_size)
test_size = int(0.125 * dataset_size)

temp_ds = train_ds.shuffle(1000, seed=42)
train_ds = temp_ds.take(train_size)
test_ds = temp_ds.skip(train_size)

In [ ]:
n_classes = 2
def resnet_v2_preprocess_x(x, y):
  x = tf.keras.applications.resnet_v2.preprocess_input(x)
  return x,y
resnet_v2_ds_train = train_ds.map(resnet_v2_preprocess_x)
resnet_v2_ds_val = validation_ds.map(resnet_v2_preprocess_x)
resnet_v2_ds_test = test_ds.map(resnet_v2_preprocess_x)

In [ ]:
dataset_size = resnet_v2_ds_train.cardinality().numpy()
dataset_size

72

In [ ]:
tf.keras.backend.clear_session()
base_model = keras.applications.ResNet152V2(weights="imagenet",
                                            include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(resnet_v2_ds_train,
                    validation_data=resnet_v2_ds_val,
                    epochs=10)

Epoch 1/10
72/72 [==============================] - 202s 676ms/step - loss: 4.2875 - accuracy: 0.7768 - F1_score: 0.2311 - Precision: 0.1324 - Recall: 0.9861 - val_loss: 3.8882 - val_accuracy: 0.6062 - val_F1_score: 0.2610 - val_Precision: 0.1523 - val_Recall: 1.0000
Epoch 2/10
72/72 [==============================] - 32s 317ms/step - loss: 2.4546 - accuracy: 0.7866 - F1_score: 0.2456 - Precision: 0.1416 - Recall: 1.0000 - val_loss: 2.3036 - val_accuracy: 0.7426 - val_F1_score: 0.2613 - val_Precision: 0.1517 - val_Recall: 1.0000
Epoch 3/10
72/72 [==============================] - 29s 315ms/step - loss: 1.5768 - accuracy: 0.8040 - F1_score: 0.2453 - Precision: 0.1420 - Recall: 1.0000 - val_loss: 2.8315 - val_accuracy: 0.7890 - val_F1_score: 0.2659 - val_Precision: 0.1544 - val_Recall: 1.0000
Epoch 4/10
72/72 [==============================] - 29s 315ms/step - loss: 1.5680 - accuracy: 0.8044 - F1_score: 0.2349 - Precision: 0.1348 - Recall: 0.9861 - val_loss: 2.7011 - val_accuracy: 0.5270

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy', F1_score, Precision, Recall])
history = model.fit(resnet_v2_ds_train,
                    validation_data=resnet_v2_ds_val,
                    epochs=20)

Epoch 1/20
72/72 [==============================] - 85s 810ms/step - loss: 0.9350 - accuracy: 0.8541 - F1_score: 0.2380 - Precision: 0.1363 - Recall: 1.0000 - val_loss: 62524.8438 - val_accuracy: 0.8469 - val_F1_score: 0.2657 - val_Precision: 0.1544 - val_Recall: 1.0000
Epoch 2/20
72/72 [==============================] - 61s 764ms/step - loss: 0.5367 - accuracy: 0.8646 - F1_score: 0.2317 - Precision: 0.1328 - Recall: 0.9861 - val_loss: 101589.9922 - val_accuracy: 0.8286 - val_F1_score: 0.2615 - val_Precision: 0.1517 - val_Recall: 1.0000
Epoch 3/20
72/72 [==============================] - 61s 763ms/step - loss: 0.5170 - accuracy: 0.8543 - F1_score: 0.2427 - Precision: 0.1400 - Recall: 0.9861 - val_loss: 10280.6914 - val_accuracy: 0.8469 - val_F1_score: 0.2651 - val_Precision: 0.1544 - val_Recall: 1.0000
Epoch 4/20
72/72 [==============================] - 61s 764ms/step - loss: 0.4732 - accuracy: 0.8589 - F1_score: 0.2414 - Precision: 0.1392 - Recall: 1.0000 - val_loss: 626.6621 - val_ac

In [ ]:
score = model.evaluate(resnet_v2_ds_test)

11/11 [==============================] - 9s 239ms/step - loss: 0.3954 - accuracy: 0.8651 - F1_score: 0.2352 - Precision: 0.1349 - Recall: 1.0000
